# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,4.06,75,75,5.66,CL,1681707819
1,1,bluff,-46.6000,168.3333,17.03,68,99,2.33,NZ,1681707820
2,2,birjand,32.8663,59.2211,16.26,27,100,0.00,IR,1681707639
3,3,vaini,-21.2000,-175.2000,26.09,83,75,8.23,TO,1681707820
4,4,pangody,65.8500,74.4859,-15.46,90,100,6.50,RU,1681707821


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
# Narrow down the DataFrame to find cities that fit the ideal weather conditions
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 25) &
    (city_data_df['Humidity'] > 30) & (city_data_df['Humidity'] < 60) &
    (city_data_df['Cloudiness'] < 10) & (city_data_df['Wind Speed'] < 10)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display a random sample of the data
ideal_cities_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,abu samrah,35.3029,37.1841,20.76,34,4,0.94,SY,1681707834
83,83,suez,29.9737,32.5263,20.14,45,0,4.19,EG,1681707841
138,138,san patricio,28.0170,-97.5169,22.92,57,0,3.07,US,1681707860
291,291,naze,28.3667,129.4833,21.51,52,0,3.92,JP,1681707898
376,376,esperance,-33.8667,121.9000,22.34,54,3,5.85,AU,1681707934


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# create a new empty DataFrame to store hotel information
hotel_df = pd.DataFrame(columns=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'])

# Copy values from ideal_cities_df DataFrame to hotel_df
hotel_df['City'] = ideal_cities_df['City']
hotel_df['Country'] = ideal_cities_df['Country']
hotel_df['Lat'] = ideal_cities_df['Lat']
hotel_df['Lng'] = ideal_cities_df['Lng']
hotel_df['Humidity'] = ideal_cities_df['Humidity']

# Add an empty column for the hotel name
hotel_df['Hotel Name'] = ""

# Display a random sample of the data
hotel_df.sample(5)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
83,suez,EG,29.9737,32.5263,45,
511,murray bridge,AU,-35.1167,139.2667,41,
291,naze,JP,28.3667,129.4833,52,
52,abu samrah,SY,35.3029,37.1841,34,
376,esperance,AU,-33.8667,121.9000,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey": geoapify_key,
    "sort": "distance", 
    "filter_category_ids": "6300",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat =row["Lat"]
    lng =row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance({lng},{lat}<=(radius))"
    params["bias"] = f"proximity({lng},{lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
abu samrah - nearest hotel: No hotel found
suez - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
mehran - nearest hotel: No hotel found
aransas pass - nearest hotel: No hotel found
murray bridge - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
52,abu samrah,SY,35.3029,37.1841,34,No hotel found
83,suez,EG,29.9737,32.5263,45,No hotel found
138,san patricio,US,28.0170,-97.5169,57,No hotel found
291,naze,JP,28.3667,129.4833,52,No hotel found
376,esperance,AU,-33.8667,121.9000,54,No hotel found
457,mehran,IR,33.1222,46.1646,41,No hotel found
466,aransas pass,US,27.9095,-97.1500,57,No hotel found
511,murray bridge,AU,-35.1167,139.2667,41,No hotel found
